In [1]:
from data_provider.dataloader_CUEE import  DatasetCUEE
from exp.exp_basic import Exp_Basic
from models import Informer, Autoformer, Transformer, DLinear, Linear, NLinear
from utils.tools import EarlyStopping, adjust_learning_rate, visual, test_params_flop
from utils.metrics import metric

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch import optim

import os
import time

import warnings
import matplotlib.pyplot as plt 

import pdb

In [2]:
class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

In [3]:
root_path = "./dataset/CUEE/"
data_path = "updated_measurement_Iclr_new.csv"
seq_len   = 37
label_len = 0 
pred_len  = 4
features  = "S"
target    = "I"
embed     = 'timeF'
timeenc   = 0 if embed != 'timeF' else 1
freq      = 'h'
scale     = False

Data = DatasetCUEE
# Maybe you should change the code to subsample
data_set = Data(root_path=root_path, data_path=data_path, flag='test', size=[ seq_len, label_len, pred_len],
                features= features, target=target, scale=scale, timeenc=timeenc, freq=freq, train_only=False)

[ISL009] Total 19635 Train 13744 Test 3927
[ISL015] Total 19609 Train 13726 Test 3921
[ISL019] Total 19634 Train 13743 Test 3926
[ISL020] Total 19617 Train 13731 Test 3923
[ISL027] Total 19629 Train 13740 Test 3925
[ISL028] Total 19420 Train 13594 Test 3884
[ISL038] Total 19570 Train 13699 Test 3914
[ISL043] Total 19572 Train 13700 Test 3914
[ISL012] Total 19525 Train 13667 Test 3905
[ISL013] Total 13773 Train 9641 Test 2754
[ISL017] Total 18710 Train 13097 Test 3742
[ISL018] Total 19521 Train 13664 Test 3904
[ISL021] Total 19636 Train 13745 Test 3927
[ISL022] Total 19641 Train 13748 Test 3928
[ISL025] Total 19634 Train 13743 Test 3926
[ISL026] Total 19301 Train 13510 Test 3860
[ISL030] Total 19511 Train 13657 Test 3902
[ISL031] Total 19221 Train 13454 Test 3844
[ISL039] Total 19372 Train 13560 Test 3874
[ISL041] Total 19626 Train 13738 Test 3925
[ISL014] Total 19575 Train 13702 Test 3915
[ISL029] Total 19638 Train 13746 Test 3927
[ISL034] Total 19193 Train 13435 Test 3838
[ISL037] Tot

In [4]:
kernel_size_large = 37
decompsition_function_large = series_decomp(kernel_size_large)


kernel_size_small = 5
decompsition_function_small = series_decomp(kernel_size_small)

from torch.utils.data import Dataset, DataLoader
dataloader = DataLoader(data_set)

num_samples = len(dataloader)
 
timeseries_data_list  = []

for ind, data_ in enumerate(dataloader):

    seq_x_, seq_y_, seq_x_mark, seq_y_mark  = data_  

    seq_x = seq_x_.view(-1)
    seq_y = seq_y_.view(-1)

    if ind == num_samples:
        timeseries_data_list.append(seq_y[:])
    elif ind == 0: 
        timeseries_data_list.append(np.concatenate([seq_x[:], seq_y[:-(pred_len - 1)]]))
    else:
        timeseries_data_list.append(seq_y[:-(pred_len - 1)])
    
    
timeseries_data = np.concatenate(timeseries_data_list)

timeseries_data_num =  len(timeseries_data) 
timeseries_data = timeseries_data.reshape([1,timeseries_data_num,1]) 
timeseries_data.shape
timeseries_data = torch.from_numpy(timeseries_data)

seasonal_x, trend_x = decompsition_function_large(timeseries_data)

seasonal_x_small, trend_x_small = decompsition_function_small(seasonal_x)

timeseries_data = timeseries_data.view(-1).numpy()  
trend_x = trend_x.view(-1).numpy() 
seasonal_x = seasonal_x.view(-1).numpy()

trend_x_small = trend_x_small.view(-1).numpy() 
seasonal_x_small = seasonal_x_small.view(-1).numpy()

In [5]:
# Set the default text font size
plt.rc('font', size=16)
# Set the axes title font size
plt.rc('axes', titlesize=16)
# Set the axes labels font size
plt.rc('axes', labelsize=16)
# Set the font size for x tick labels
plt.rc('xtick', labelsize=16)
# Set the font size for y tick labels
plt.rc('ytick', labelsize=16)
# Set the legend font size
plt.rc('legend', fontsize=18)
# Set the font size of the figure title
plt.rc('figure', titlesize=20)

fig, axs = plt.subplots(nrows=4, ncols=1, sharex=True, sharey=True, figsize=(12, 12))
axs[0].plot(timeseries_data)
axs[0].set_xlim([0,1000])
axs[0].set_title("Irradiance $I$" )
axs[0].grid(True)

axs[1].plot(seasonal_x_small)
axs[1].set_xlim([0,1000])
axs[1].set_title("Hourly Residual [Kernel Hourly=%d]" % kernel_size_small)
axs[1].grid(True)

axs[2].plot(trend_x_small)
axs[2].set_xlim([0,1000])
axs[2].set_title("Daily Seasonality [Kernel Hourly=%d, Kernel Daily=%d]" % (kernel_size_small, kernel_size_large))
axs[2].grid(True)


axs[3].plot(trend_x)
axs[3].set_xlim([0,1000])
axs[3].set_title("Trend [Kernel=%d]" % kernel_size_large)
axs[3].set_xlabel("Samples")
axs[3].grid(True) 

if scale == False:
    scale_text = "WONorm"
else:
    scale_text = "Normal"

plt.savefig("time-series-Double-SmallKernel-%d-LargeKernel-%d-%s.png" % (kernel_size_small, kernel_size_large, scale_text)) 
print("time-series-Double-SmallKernel-%d-LargeKernel-%d-%s.png" % (kernel_size_small, kernel_size_large, scale_text))

time-series-Double-SmallKernel-5-LargeKernel-37-WONorm.png
